In [1]:
!pip install simpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import simpy

In [3]:
import numpy as np
import pandas as pd
import random
from collections import defaultdict

In [4]:
class Restaurant:
    def __init__(self, env, num_instances, max_wait_time_list):
        self.env = env
        self.rest_manager = simpy.PriorityResource(env, num_instances[0])
        self.cust_manager = simpy.PriorityResource(env, num_instances[1])
        self.order_manager = simpy.PriorityResource(env, num_instances[2])
        self.delivery_server = simpy.PriorityResource(env, num_instances[3])
        self.pay_server = simpy.PriorityResource(env, num_instances[4])
        self.api_port = simpy.PriorityResource(env, num_instances[5])
        self.web_port = simpy.PriorityResource(env, num_instances[6])

        self.order_phone_max_wait = max_wait_time_list[0]
        self.order_web_max_wait = max_wait_time_list[1]
        self.deliver_message_max_wait = max_wait_time_list[2]
        self.check_info_phone_max_wait = max_wait_time_list[3]
        self.check_info_web_max_wait = max_wait_time_list[4]
        self.request_deliver_max_wait = max_wait_time_list[5]
        self.track_order_max_wait = max_wait_time_list[6]
        
        self.rest_manage_service_time = 8
        self.cust_manage_service_time = 5
        self.order_manage_service_time = 6
        self.delivery_service_time = 9
        self.pay_service_time = 12
        self.api_port_service_time = 2
        self.web_port_service_time = 3


    def api_port_process(self, customer):
        yield self.env.timeout(np.random.exponential(self.api_port_service_time))

    def web_port_process(self, customer):
        yield self.env.timeout(np.random.exponential(self.web_port_service_time))
    
    def cust_manage_process(self, customer):
        yield self.env.timeout(np.random.exponential(self.cust_manage_service_time))

    def rest_manage_process(self, customer):
        yield self.env.timeout(np.random.exponential(self.rest_manage_service_time))

    def order_manage_process(self, customer):
        yield self.env.timeout(np.random.exponential(self.order_manage_service_time))
    
    def delivery_service_process(self, customer):
        yield self.env.timeout(np.random.exponential(self.delivery_service_time))

    def pay_service_process(self, customer):
        yield self.env.timeout(np.random.exponential(self.pay_service_time))


# Servers

In [5]:
def api_port_request(env, customer, restaurant, priority, request_type, 
                     waiting_threshold):
    
    with restaurant.api_port.request(priority=priority) as request:
        queue_in = env.now
        time_queue['Api Port'].append(queue_in)
        len_queue['Api Port'].append(len(restaurant.api_port.queue))
        
        yield request

        queue_out = env.now
        time_queue['Api Port'].append(queue_out)
        len_queue['Api Port'].append(len(restaurant.api_port.queue))

        time_in_queue = queue_out - queue_in
        in_queue['Api Port'].append(time_in_queue)
        in_queue_request[request_type].append(time_in_queue)
        
        yield env.process(restaurant.api_port_process(customer))

In [6]:
def web_port_request(env, customer, restaurant, priority, request_type, 
                     waiting_threshold):
    
    with restaurant.web_port.request(priority=priority) as request:
        queue_in = env.now
        time_queue['Web Port'].append(queue_in)
        len_queue['Web Port'].append(len(restaurant.web_port.queue))
        
        yield request
        
        queue_out = env.now
        time_queue['Web Port'].append(queue_out)
        len_queue['Web Port'].append(len(restaurant.web_port.queue))

        time_in_queue = queue_out - queue_in
        in_queue['Web Port'].append(time_in_queue)
        in_queue_request[request_type].append(time_in_queue)

        yield env.process(restaurant.web_port_process(customer))

In [7]:
def order_manager_request(env, customer, restaurant, priority, request_type, 
                          waiting_threshold):
    
    with restaurant.order_manager.request(priority=priority) as request:
        queue_in = env.now
        time_queue['Order Manager'].append(queue_in)
        len_queue['Order Manager'].append(len(restaurant.order_manager.queue))
        
        yield request
        
        queue_out = env.now
        time_queue['Order Manager'].append(queue_out)
        len_queue['Order Manager'].append(len(restaurant.order_manager.queue))

        time_in_queue = queue_out - queue_in
        in_queue['Order Manager'].append(time_in_queue)
        in_queue_request[request_type].append(time_in_queue)
        
        yield env.process(restaurant.order_manage_process(customer))

In [8]:
def pay_server_request(env, customer, restaurant, priority, request_type, 
                       waiting_threshold):
    
    with restaurant.pay_server.request(priority=priority) as request:
        queue_in = env.now
        time_queue['Pay Server'].append(queue_in)
        len_queue['Pay Server'].append(len(restaurant.pay_server.queue))
        
        yield request
        
        queue_out = env.now
        time_queue['Pay Server'].append(queue_out)
        len_queue['Pay Server'].append(len(restaurant.pay_server.queue))
        
        time_in_queue = queue_out - queue_in
        in_queue['Pay Server'].append(time_in_queue)
        in_queue_request[request_type].append(time_in_queue)

        yield env.process(restaurant.pay_service_process(customer))

In [9]:
def cust_manager_request(env, customer, restaurant, priority, request_type, 
                         waiting_threshold):
    
    with restaurant.cust_manager.request(priority=priority) as request:
        queue_in = env.now
        time_queue['Customer Manager'].append(queue_in)
        len_queue['Customer Manager'].append(len(restaurant.cust_manager.queue))
        
        yield request
        
        queue_out = env.now
        time_queue['Customer Manager'].append(queue_out)
        len_queue['Customer Manager'].append(len(restaurant.cust_manager.queue))

        time_in_queue = queue_out - queue_in
        in_queue['Customer Manager'].append(time_in_queue)
        in_queue_request[request_type].append(time_in_queue)
        
        yield env.process(restaurant.cust_manage_process(customer))

In [10]:
def delivery_server_request(env, customer, restaurant, priority, request_type, 
                            waiting_threshold):
    
    with restaurant.delivery_server.request(priority=priority) as request:
        queue_in = env.now
        time_queue['Delivery Server'].append(queue_in)
        len_queue['Delivery Server'].append(len(restaurant.delivery_server.queue))
        
        yield request
        
        queue_out = env.now
        time_queue['Delivery Server'].append(queue_out)
        len_queue['Delivery Server'].append(len(restaurant.delivery_server.queue))

        time_in_queue = queue_out - queue_in
        in_queue['Delivery Server'].append(time_in_queue)
        in_queue_request[request_type].append(time_in_queue)

        yield env.process(restaurant.delivery_service_process(customer))

In [11]:
def rest_manager_request(env, customer, restaurant, priority, request_type,
                         waiting_threshold):

    with restaurant.rest_manager.request(priority=priority) as request:
        queue_in = env.now
        time_queue['Restaurant Manager'].append(queue_in)
        len_queue['Restaurant Manager'].append(len(restaurant.rest_manager.queue))

        yield request
        queue_out = env.now
        time_queue['Restaurant Manager'].append(queue_out)
        len_queue['Restaurant Manager'].append(len(restaurant.rest_manager.queue))
        
        time_in_queue = queue_out - queue_in
        in_queue['Restaurant Manager'].append(time_in_queue)
        in_queue_request[request_type].append(time_in_queue)
        
        yield env.process(restaurant.rest_manage_process(customer))

# Requests

In [12]:
def order_phone(env, customer, restaurant):
    arrival_time = env.now
    priority = 1
    waiting_threshold = env.timeout(restaurant.order_phone_max_wait)

    yield env.process(api_port_request(env, customer, restaurant, priority,
                                       "Order Phone", waiting_threshold))

    yield env.process(order_manager_request(env, customer, restaurant, priority,
                                            "Order Phone", waiting_threshold))

    yield env.process(pay_server_request(env, customer, restaurant, priority,
                                         "Order Phone", waiting_threshold))

In [13]:
def order_web(env, customer, restaurant):
    arrival_time = env.now
    priority = 1
    waiting_threshold = env.timeout(restaurant.order_web_max_wait)

    yield env.process(web_port_request(env, customer, restaurant, priority,
                                       "Order Web", waiting_threshold))

    yield env.process(order_manager_request(env, customer, restaurant, priority,
                                            "Order Web", waiting_threshold))

    yield env.process(pay_server_request(env, customer, restaurant, priority,
                                         "Order Web", waiting_threshold))

In [14]:
def delivery_message(env, customer, restaurant):
    arrival_time = env.now
    priority = 2
    waiting_threshold = env.timeout(restaurant.deliver_message_max_wait)

    yield env.process(api_port_request(env, customer, restaurant, priority,
                                       "Delivery Message", waiting_threshold))

    yield env.process(cust_manager_request(env, customer, restaurant, priority,
                                           "Delivery Message", waiting_threshold))

    yield env.process(delivery_server_request(env, customer, restaurant, priority,
                                              "Delivery Message", waiting_threshold))

In [15]:
def rest_check_info_phone(env, customer, restaurant):
    arrival_time = env.now
    priority = 2
    waiting_threshold = env.timeout(restaurant.check_info_phone_max_wait)

    yield env.process(api_port_request(env, customer, restaurant, priority,
                                       "Check Info Phone", waiting_threshold))
    
    yield env.process(rest_manager_request(env, customer, restaurant, priority,
                                           "Check Info Phone", waiting_threshold))

In [16]:
def rest_check_info_web(env, customer, restaurant):
    arrival_time = env.now
    priority = 2
    waiting_threshold = env.timeout(restaurant.check_info_web_max_wait)

    yield env.process(web_port_request(env, customer, restaurant, priority,
                                       "Check Info Web", waiting_threshold))

    yield env.process(rest_manager_request(env, customer, restaurant, priority,
                                           "Check Info Web", waiting_threshold))

In [17]:
def request_delivery(env, customer, restaurant):
    arrival_time = env.now
    priority = 1
    waiting_threshold = env.timeout(restaurant.request_deliver_max_wait)

    yield env.process(web_port_request(env, customer, restaurant, priority,
                                       "Request Delivery", waiting_threshold))

    yield env.process(rest_manager_request(env, customer, restaurant, priority,
                                           "Request Delivery", waiting_threshold))

    yield env.process(delivery_server_request(env, customer, restaurant, priority, 
                                              "Request Delivery", waiting_threshold))

In [18]:
def track_order(env, customer, restaurant):
    arrival_time = env.now
    priority = 2
    waiting_threshold = env.timeout(restaurant.track_order_max_wait)

    yield env.process(api_port_request(env, customer, restaurant, priority,
                                       "Track Order", waiting_threshold))

    yield env.process(order_manager_request(env, customer, restaurant, priority,
                                            "Track Order", waiting_threshold))

# Run Simulation

In [19]:
def go_to_restaurant(env, customer, restaurant):
        random_num = random.randint(1, 101)

        if (1 <= random_num <= 20):
            yield env.process(order_phone(env, customer, restaurant))
        elif (20 < random_num <= 30):
            yield env.process(order_web(env, customer, restaurant))
        elif (30 < random_num <= 35):
            yield env.process(delivery_message(env, customer, restaurant))
        elif (35 < random_num <= 60):
            yield env.process(rest_check_info_phone(env, customer, restaurant))
        elif (60 < random_num <= 75):
            yield env.process(rest_check_info_web(env, customer, restaurant))
        elif (75 < random_num <= 95):
            yield env.process(request_delivery(env, customer, restaurant))
        else:
            yield env.process(track_order(env, customer, restaurant))

In [20]:
def run_restaurant(env, num_instances, arrival_rate, max_wait_time_list):
    restaurant = Restaurant(env, num_instances, max_wait_time_list)

    customer = 0
    env.process(go_to_restaurant(env, customer, restaurant))

    while True:
        yield env.timeout(arrival_rate)
        
        customer += 1
        env.process(go_to_restaurant(env, customer, restaurant))

# Test

In [21]:
# test 1
num_instances = [1, 1, 1, 2, 5, 3, 2]
arrival_rate = 1/30
simulation_time = 1000
max_wait_time_list = [25, 30, 25, 30, 30, 40, 20]

In [22]:
# test 2
num_instances = [2, 3, 1, 1, 1, 6, 6]
arrival_rate = 1/30
simulation_time = 1000
max_wait_time_list = [25, 30, 25, 30, 30, 40, 20]

In [40]:
# test 3
num_instances = [4, 1, 4, 4, 4, 10, 10]
arrival_rate = 1/30
simulation_time = 1000
max_wait_time_list = [25, 30, 25, 30, 30, 40, 20]

In [81]:
# Improvement
num_instances = [30, 10, 30, 30, 30, 30, 30]
arrival_rate = 1/30
simulation_time = 1000
max_wait_time_list = [25, 30, 25, 30, 30, 40, 20]

In [82]:
in_queue = defaultdict(list)
time_queue, len_queue = defaultdict(list), defaultdict(list)
in_queue_request = defaultdict(list)

In [83]:
env = simpy.Environment()
env.process(run_restaurant(env, num_instances, arrival_rate, max_wait_time_list))
env.run(until=simulation_time)

In [84]:
def get_df_server(time_queue, len_queue, server):
    df1 = pd.DataFrame(time_queue[server], columns=['time'])
    df2 = pd.DataFrame(len_queue[server], columns=['len'])
    df_server = pd.concat([df1, df2], axis=1)
    return df_server

In [85]:
def get_avg_queue(df_server):
    df_server['delta time'] = df_server['time'].shift(-1) - df_server['time']
    df_server = df_server[0:-1]
    try:
        avg = np.average(df_server['len'], weights=df_server['delta time'])
    except ZeroDivisionError:
        avg = 0
    return avg

In [86]:
def get_server_utilization(df_server):
    sum_server_free = df_server[df_server['len'] == 0]['delta time'].sum()
    sum_server_free += (simulation_time - df_server['time'].iloc[-1] 
                        if df_server['len'].iloc[-1] == 0 else 0)
    first_event = df_server['time'].iloc[0]
    sum_server_free = sum_server_free + first_event
    utilization = round((1 - sum_server_free / simulation_time) * 100, 2)
    return utilization

In [87]:
df_server = {}
for server in sorted(time_queue.keys()):
    df_server[server] = get_df_server(time_queue, len_queue, server)

avg_queue_len = {'Api Port': 0, 
                'Web Port': 0, 
                'Order Manager': 0, 
                'Pay Server': 0, 
                'Customer Manager': 0, 
                'Delivery Server': 0, 
                'Restaurant Manager': 0
                }
for server in sorted(len_queue.keys()):
    avg_queue_len[server] = get_avg_queue(df_server[server])

avg_delay = {'Api Port': 0, 
                'Web Port': 0, 
                'Order Manager': 0, 
                'Pay Server': 0, 
                'Customer Manager': 0, 
                'Delivery Server': 0, 
                'Restaurant Manager': 0
                }
for key in sorted(in_queue.keys()):
    avg_delay[key] = np.mean(in_queue[key])

server_utilization = {'Api Port': 0, 
                'Web Port': 0, 
                'Order Manager': 0, 
                'Pay Server': 0, 
                'Customer Manager': 0, 
                'Delivery Server': 0, 
                'Restaurant Manager': 0
                }
for server in sorted(df_server.keys()):
    server_utilization[server] = get_server_utilization(df_server[server])

In [88]:
pd.DataFrame({"Average Queue Length": list(avg_queue_len.values()),
              "Average Waiting Time": list(avg_delay.values()), 
              "Server Utilization": list(server_utilization.values())}, avg_queue_len.keys())

,Average Queue Length,Average Waiting Time,Server Utilization
Api Port,868.554336,49.160177,99.60
Web Port,1684.261087,40.034030,99.42
Order Manager,2751.333593,226.082159,99.47
Pay Server,1181.597320,232.411595,98.38
Customer Manager,0.242314,0.192928,8.13
Delivery Server,820.742160,60.233669,97.97
Restaurant Manager,4644.731893,188.307207,99.65


In [89]:
avg_delay_request = {"Order Phone": 0, 
                     "Order Web": 0, 
                     "Delivery Message": 0, 
                     "Check Info Phone": 0, 
                     "Check Info Web": 0, 
                     "Request Delivery": 0, 
                     "Track Order": 0
                     }
for key in sorted(in_queue_request.keys()):
    avg_delay_request[key] = np.mean(in_queue_request[key])

pd.DataFrame({"Average Waiting Time": list(avg_delay_request.values())}, avg_delay_request.keys())

,Average Waiting Time
Order Phone,103.796069
Order Web,105.246588
Delivery Message,40.931966
Check Info Phone,81.587477
Check Info Web,410.425440
Request Delivery,69.692326
Track Order,81.922458


In [90]:
num_requests = sum([len(requests) for requests in in_queue_request.values()])
total_avg_waiting_time = sum([sum(value) for value in in_queue_request.values()])/num_requests
print(f"Total Number of Requests: {num_requests} with Average of {total_avg_waiting_time} Waiting Time")

Total Number of Requests: 40747 with Average of 92.16793384867321 Waiting Time
